[![UAEM](https://www.uaem.mx/fcaei/images/uaem.png)](https://www.uaem.mx/fcaei/moca.html)
[![Pandas](https://img.shields.io/badge/Pandas-2.0+-blue.svg)](https://pandas.pydata.org/)
[![Scikit-learn](https://img.shields.io/badge/Scikit--learn-1.2+-1560BD.svg)](https://scikit-learn.org/stable/)
[![Statsmodels](https://img.shields.io/badge/Statsmodels-0.14+-8A2BE2.svg)](https://www.statsmodels.org/stable/index.html)
[![Python 3.9+](https://img.shields.io/badge/Python-3.9+-green.svg)](https://www.python.org/downloads/release/python-390/)

# Proyecto: Análisis Predictivo y de Series de Tiempo de Contaminantes Atmosféricos

**Autor:** [Emiliano Rodriguez Villegas](https://github.com/EmilianoRdzV)
**Fecha:** 14 de Junio de 2025
**Versión:** 1.0

---

## Introducción y Objetivos

Este notebook realiza un análisis multifacético sobre un conjunto de datos de seis contaminantes criterio, correspondientes al mes de enero de 2019. El proyecto se centra en dos áreas principales de la ciencia de datos:

1.  **Modelado Predictivo:** Se construirán y evaluarán modelos de regresión para estimar la concentración de ciertos contaminantes (como PM2.5 y O3) utilizando otros gases como variables de entrada.
2.  **Análisis de Series de Tiempo:** Se analizará el comportamiento temporal de un contaminante específico para descomponer su serie, identificar su tendencia y observar los patrones subyacentes una vez eliminado el componente estacional.

---

### Índice del Notebook

1.  [**Fase 1: Preparación y Análisis Exploratorio**](#fase-1)
    * [1.1. Carga de Librerías y Datos](#1-1)
    * [1.2. Limpieza y Verificación de Datos](#1-2)
    * [1.3. Análisis Exploratorio y Matriz de Correlación](#1-3)
2.  [**Fase 2: Modelado Predictivo (Regresión)**](#fase-2)
    * [2.1. Modelo 1: Predicción de Partículas PM10 / PM2.5](#2-1)
    * [2.2. Modelo 2: Predicción de Ozono (O3)](#2-2)
    * [2.3. Evaluación de Modelos (Métricas y Análisis de valor-t)](#2-3)
3.  [**Fase 3: Análisis de Series de Tiempo**](#fase-3)
    * [3.1. Selección y Visualización de la Serie Temporal](#3-1)
    * [3.2. Descomposición de la Serie](#3-2)
    * [3.3. Observaciones de la Serie Desestacionalizada](#3-3)
4.  [**Fase 4: Conclusiones Finales**](#fase-4)
    * [4.1. Resumen de Hallazgos](#4-1)
    * [4.2. Pasos Futuros](#4-2)

## <a id="fase-1"></a>1. Fase 1: Preparación y Análisis Exploratorio

En esta fase inicial, cargaremos todas las herramientas necesarias y nuestro conjunto de datos. Realizaremos una limpieza básica y una exploración para entender la estructura y las características principales de los datos antes de pasar al modelado.

### <a id="1-1"></a>1.1. Carga de Librerías y Datos

Comenzamos importando todas las librerías de Python que utilizaremos a lo largo del proyecto. Posteriormente, cargaremos el conjunto de datos desde el archivo local a un DataFrame de `pandas` que llamaremos `contaminantes`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose


rutaArchivo = '../Data/contaminantes.xlsx' 
contaminantes = pd.read_excel(rutaArchivo)

display(contaminantes.head())

,DATE,CO,NO2,O3,SO2,PM2.5,PM10
0,2019-01-01,0.88,0.011,0.021,0.003,38,59.0
1,2019-01-01,0.82,0.012,0.018,0.002,46,62.0
2,2019-01-01,0.75,0.009,0.018,0.002,31,45.0
3,2019-01-01,0.67,0.006,0.020,0.002,18,29.0
4,2019-01-01,0.61,0.004,0.021,0.002,13,23.0


### <a id="1-2"></a>1.2. Limpieza y Verificación de Datos

Una vez cargados los datos, el siguiente paso crítico es la limpieza. Verificaremos la existencia de valores nulos (NaN) y filas duplicadas que puedan afectar la calidad de nuestro análisis y modelos.

**Estrategia de Limpieza:**
* **Valores Nulos:** Para los valores numéricos faltantes, utilizaremos el método de **interpolación lineal**. Esta técnica es ideal para datos de series temporales (como mediciones de sensores), ya que estima los valores faltantes basándose en los puntos anterior y posterior, asumiendo una progresión lógica.
* **Duplicados:** Se eliminará cualquier fila que sea una copia exacta de otra para evitar redundancia en los datos.

In [16]:
# Buscamos valores nulos/duplicados que puedan interferir en el analisiss 
totalNulos = contaminantes.isnull().sum().sum()
totalDuplicados = contaminantes.duplicated().sum()

print("\n--- Resumen final de los tipos de datos ---")
contaminantes.info()


--- Resumen final de los tipos de datos ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    742 non-null    datetime64[ns]
 1   CO      742 non-null    float64       
 2   NO2     742 non-null    float64       
 3   O3      742 non-null    float64       
 4   SO2     742 non-null    float64       
 5   PM2.5   742 non-null    int64         
 6   PM10    742 non-null    float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 40.7 KB
